__This lists the major steps to building and training a logistics regression model for classification__

* Read in the .mat file, view the data and process the data worthy for building and training our classification model.
* build and train our model with training datasets.
* Test our model for accuracy using testing datasets.
* conduct Cross validation of our model using "leave-one-out cross-validation" & "k-fo;d corss-validation" methods.
* conduct Feature importance using both "recursive feature elimination" & "permutation testing".


 __Each step is repeated for the number of classifier models required for classifying the datasets__ 


__1. Read the .mat file, view the data and process the datasets for classification__ 

In [1]:
# Import the .mat file using laodmat module from scipy.io

from scipy.io import loadmat

input_data = loadmat('SFC_features_alldays_taskposneg.mat')

#check the labels of the features of the imported dataset
input_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'feature_IDs_taskposneg', 'feature_IDs_taskposneg_char', 'info_features_alldays', 'sfc_taskposneg_ComBat_day1', 'sfc_taskposneg_ComBat_day2', 'sfc_taskposneg_ComBat_day3'])

In [2]:
#********************************************************************************************************************#
# the main features required for generating our training and tresting datasets are:
#'feature_IDs_attention';          // required for features importance
#'feature_IDs_attention_char';     // data type of features _IDs
#'info_features_alldays';          // metadata about the size of datasets 
#'sfc_attention_ComBat_day1';       // day1 data
#'sfc_attention_ComBat_day2';       // day2 data
#'sfc_attention_ComBat_day3'        // day 3 data
#******************************************************************************************************************#

# we write the features for use into variables for further data cleaning

feature_ids = input_data['feature_IDs_taskposneg']
feature_ids_char = input_data['feature_IDs_taskposneg_char']
features_info = input_data['info_features_alldays']
day1_data = input_data['sfc_taskposneg_ComBat_day1']
day2_data = input_data['sfc_taskposneg_ComBat_day2']
day3_data = input_data['sfc_taskposneg_ComBat_day3']

#we can view the features information for all days
print(features_info)


['sfc features = Nx36, where N are the features and 36 are the subjects\nfeature_IDs provides the connection coordinates in the pp272 space (within the given networks) corresponding to these N features\nComBat = site harmonization performed using the ComBat technique\n']


In [3]:
# N x 36 is the size of the dataset for each day, to view the actual size of N, we use panda and numpy libraries
import pandas as pd
import numpy as np

feature_ids = input_data['feature_IDs_taskposneg']
f_ids = pd.DataFrame(feature_ids)
feature_ids = f_ids.to_numpy()
print("feature_ids size",feature_ids.shape)

feature_ids_char = input_data['feature_IDs_taskposneg_char']
f_ids_char = pd.DataFrame(feature_ids_char)
feature_ids_char = f_ids_char.to_numpy()
print("feature_ids_char size",feature_ids_char.shape)

day1_data = input_data['sfc_taskposneg_ComBat_day1']
d1 = pd.DataFrame(day1_data)
day1_data = d1.to_numpy()
print("day1_data size",day1_data.shape)

day2_data = input_data['sfc_taskposneg_ComBat_day2']
d2 = pd.DataFrame(day2_data)
day2_data = d2.to_numpy()
print("day2_data size",day2_data.shape)

day3_data = input_data['sfc_taskposneg_ComBat_day3']
d3 = pd.DataFrame(day3_data)
day3_data = d3.to_numpy()
print("day3_data size",day3_data.shape)


feature_ids size (4503, 2)
feature_ids_char size (4503, 1)
day1_data size (4503, 36)
day2_data size (4503, 36)
day3_data size (4503, 36)


In [4]:
# the result above shows that N is 190 which is the number of features and 36 is the numer of subjects
# the required datasets for training and testing need to be transposed to conform to nsamples x nfeatures scope
#****************************************************************************************************************
data1 = np.transpose(day1_data)
print("data1 size", data1.shape)

data2 = np.transpose(day2_data)
print("data2 size", data2.shape)

data3 = np.transpose(day3_data)
print("data3 size", data3.shape)


data1 size (36, 4503)
data2 size (36, 4503)
data3 size (36, 4503)


In [5]:
# split the data into training and testing sets for our model classifiers using test_train_split module in sklearn
# we first combine the datasets in the following way
# data12 = data1 + data2 | data13 = data1 + data3 | data23 = data2 + data3;
#*********************************************************************************************************
from sklearn.model_selection import train_test_split

#generate training and testing datasets for classifier for day1 and day2 data
# Y data has to be generated using numpy for y_train and y_test data as well
data12 = np.concatenate((data1, data2), axis = 0)
y1 = np.zeros(data1.shape[0])
y2 = np.ones(data2.shape[0])
y12 = np.concatenate((y1, y2), axis = 0)

#generate training and testing datasets for classifier for day2 and day3 data
# Y data has to be generated using numpy for y_train and y_test data as well
data23 = np.concatenate((data2, data3), axis = 0)
y2 = np.zeros(data2.shape[0])
y3 = np.ones(data3.shape[0])
y23 = np.concatenate((y2, y3), axis = 0)


#generate training and testing datasets for classifier for day1 and day3 data
# Y data has to be generated using numpy for y_train and y_test data as well
data13 = np.concatenate((data1, data3), axis = 0)
y1 = np.zeros(data1.shape[0])
y3 = np.ones(data3.shape[0])
y13 = np.concatenate((y1, y3), axis = 0)


# split concatenated data into training and testing sets
X12_train, X12_test, Y12_train, Y12_test = train_test_split(data12, y12, test_size = 0.2, random_state = 10)
print('x12_train size', X12_train.shape)
print('y12_train size', Y12_train.shape)
print('x12_test size', X12_test.shape)
print('y12_test size', Y12_test.shape)

X13_train, X13_test, Y13_train, Y13_test = train_test_split(data13, y13, test_size = 0.2, random_state = 10)
print('x13_train size', X13_train.shape)
print('y13_train size', Y13_train.shape)
print('x13_test size', X13_test.shape)
print('y13_test size', Y13_test.shape)

X23_train, X23_test, Y23_train, Y23_test = train_test_split(data23, y23, test_size = 0.2, random_state = 10)
print('x23_train size', X23_train.shape)
print('y23_train size', Y23_train.shape)
print('x23_test size', X23_test.shape)
print('y23_test size', Y23_test.shape)


x12_train size (57, 4503)
y12_train size (57,)
x12_test size (15, 4503)
y12_test size (15,)
x13_train size (57, 4503)
y13_train size (57,)
x13_test size (15, 4503)
y13_test size (15,)
x23_train size (57, 4503)
y23_train size (57,)
x23_test size (15, 4503)
y23_test size (15,)


__2. Build model classifiers for each predictions, and test its predictive accuracy__

In [6]:
# the model classifier to be built will be based on logistics regression with regularization capability.
from sklearn.linear_model import LogisticRegression

# build classfier12
classifier_12 = LogisticRegression(penalty='l2', dual=False, tol=0.00001, C=1.0, fit_intercept=True, 
                                  intercept_scaling=1, class_weight='balanced', solver='newton-cg', max_iter=1000,
                                  multi_class='auto')

#build classifier23
classifier_13 = LogisticRegression(penalty='l2', dual=False, tol=0.00001, C=1.0, fit_intercept=True, 
                                  intercept_scaling=1, class_weight='balanced', solver='newton-cg', max_iter=1000,
                                  multi_class='auto')

classifier_23 = LogisticRegression(penalty='l2', dual=False, tol=0.00001, C=1.0, fit_intercept=True, 
                                  intercept_scaling=1, class_weight='balanced', solver='newton-cg', max_iter=1000,
                                  multi_class='auto')

In [7]:
# train the classifier model built 

# classifier12 training
classifier_12.fit(X12_train, Y12_train)

# classifier13 training
classifier_13.fit(X13_train, Y13_train)

#classifier23 training
classifier_23.fit(X23_train, Y23_train)

LogisticRegression(class_weight='balanced', max_iter=1000, solver='newton-cg',
                   tol=1e-05)

__3. Test the prediction accuracy of our classifiers__

In [8]:
#Test the classifiers for prediction accuracy and number of positive and negative predictions
from sklearn import metrics
from sklearn.metrics import confusion_matrix

#classifier12 accuracy on test set X12
y12_pred = classifier_12.predict(X12_test)
print('prediction Accuracy of classifier12 on test set X12: {:.5f}%'.format(classifier_12.score(X12_test, Y12_test)*100))

#classifier13 accuracy on test set X13
y13_pred = classifier_13.predict(X13_test)
print('prediction Accuracy of classifier13 on test set X13: {:.5f}%'.format(classifier_13.score(X13_test, Y13_test)*100))

#classifier23 accuracy on test set X23
y23_pred = classifier_23.predict(X23_test)
print('prediction Accuracy of classifier23 on test set X23: {:.5f}%'.format(classifier_23.score(X23_test, Y23_test)*100))


prediction Accuracy of classifier12 on test set X12: 46.66667%
prediction Accuracy of classifier13 on test set X13: 46.66667%
prediction Accuracy of classifier23 on test set X23: 46.66667%


In [9]:
# check the number of positive and negative predictions

#classifier12 number of predictions
matrix_12 = confusion_matrix(Y12_test, y12_pred)
print('prediction matrix for classifier12: ', matrix_12)

#classifier13 number of predictions
matrix_13 = confusion_matrix(Y13_test, y13_pred)
print('prediction matrix for classifier13: ', matrix_13)

#classifier23 number of predictions
matrix_23 = confusion_matrix(Y23_test, y23_pred)
print('prediction matrix for classifier23: ', matrix_23)


prediction matrix for classifier12:  [[4 4]
 [4 3]]
prediction matrix for classifier13:  [[3 5]
 [3 4]]
prediction matrix for classifier23:  [[4 4]
 [4 3]]


__4. Cross-validation for the evaluation of our classifiers using 'leave-one-out' & 'k-fold' methods__

In [10]:
# Cross-validation using Leave-One-Out method

from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score
from statistics import mean
from statistics import stdev

# carry out cross validation on classifier12 model
data12 = np.concatenate((data1, data2), axis = 0)
y1 = np.zeros(data1.shape[0])
y2 = np.ones(data1.shape[0])
y = np.concatenate((y1, y2), axis = 0)

loo_12 = LeaveOneOut()
loo_12_result=[]
for train_index, test_index in loo_12.split(data12):
    x12l_train, x12l_test = data12[train_index], data12[test_index]
    y12l_train, y12l_test = y[train_index], y[test_index]
    classifier_12.fit(x12l_train, y12l_train)
    y12l_pred = classifier_12.predict(x12l_test)
    loo_12_result.append(accuracy_score(y12l_test, y12l_pred))

# accumulated loo result    
acc_result_12 = loo_12_result

#calculate the standard deviation and mean of the accuracy
print("the mean accuracy is: {}".format(mean(acc_result_12)))
print("the standard deviation of the accuracy is: {}".format(stdev(acc_result_12)))

the mean accuracy is: 0.4305555555555556
the standard deviation of the accuracy is: 0.49862879271703475


In [11]:
# carry out cross validation on classifier13 model
data13 = np.concatenate((data1, data3), axis = 0)
y1 = np.zeros(data1.shape[0])
y3 = np.ones(data1.shape[0])
y = np.concatenate((y1, y3), axis = 0)

loo_13 = LeaveOneOut()
loo_13_result=[]
for train_index, test_index in loo_13.split(data13):
    x13l_train, x13l_test = data13[train_index], data13[test_index]
    y13l_train, y13l_test = y[train_index], y[test_index]
    classifier_13.fit(x13l_train, y13l_train)
    y13l_pred = classifier_13.predict(x13l_test)
    loo_13_result.append(accuracy_score(y13l_test, y13l_pred))

# accumulated loo result    
acc_result_13 = loo_13_result

#calculate the standard deviation and mean of the accuracy
print("the mean accuracy is: {}".format(mean(acc_result_13)))
print("the standard deviation of the accuracy is: {}".format(stdev(acc_result_13)))

the mean accuracy is: 0.5555555555555556
the standard deviation of the accuracy is: 0.5003910833605344


In [12]:
# carry out cross validation on classifier13 model
data23 = np.concatenate((data2, data3), axis = 0)
y2 = np.zeros(data1.shape[0])
y3 = np.ones(data1.shape[0])
y = np.concatenate((y2, y3), axis = 0)

loo_23 = LeaveOneOut()
loo_23_result=[]
for train_index, test_index in loo_23.split(data23):
    x23l_train, x23l_test = data23[train_index], data23[test_index]
    y23l_train, y23l_test = y[train_index], y[test_index]
    classifier_23.fit(x13l_train, y23l_train)
    y23l_pred = classifier_23.predict(x23l_test)
    loo_23_result.append(accuracy_score(y23l_test, y23l_pred))

# accumulated loo result    
acc_result_23 = loo_23_result

#calculate the standard deviation and mean of the accuracy
print("the mean accuracy is: {}".format(mean(acc_result_23)))
print("the standard deviation of the accuracy is: {}".format(stdev(acc_result_23)))

the mean accuracy is: 0.6666666666666666
the standard deviation of the accuracy is: 0.47471266327754136


In [13]:
# cross validation for the evaluation of classifiers using k-fold method
from sklearn.model_selection import KFold

#kfold is implemented for n_split = 9

# classifier12 model validation 
kf_12 = KFold(n_splits=9, shuffle=True, random_state=32)

data12 = np.concatenate((data1, data2), axis = 0)
y1 = np.zeros(data1.shape[0])
y2 = np.ones(data1.shape[0])
y = np.concatenate((y1, y2), axis = 0)

kf_12_result = []
for train_index, test_index in kf_12.split(data12):
    x12k_train, x12k_test = data12[train_index], data12[test_index]
    y12k_train, y12k_test = y[train_index], y[test_index]
    classifier_12.fit(x12k_train, y12k_train)
    y12k_pred = classifier_12.predict(x12k_test)
    kf_12_result.append(accuracy_score(y12k_test, y12k_pred))

# accumulated kfold result    
acc_result_12 = kf_12_result

#calculate the standard deviation and mean of the accuracy
print("the mean accuracy is: {}".format(mean(acc_result_12)))
print("the standard deviation of the accuracy is: {}".format(stdev(acc_result_12)))




the mean accuracy is: 0.4166666666666667
the standard deviation of the accuracy is: 0.13975424859373686


In [14]:
# cross-validation for classifier13
kf_13 = KFold(n_splits=9, shuffle=True, random_state=32)

data13 = np.concatenate((data1, data3), axis = 0)
y1 = np.zeros(data1.shape[0])
y3 = np.ones(data1.shape[0])
y = np.concatenate((y1, y3), axis = 0)

kf_13_result = []
for train_index, test_index in kf_13.split(data13):
    x13k_train, x13k_test = data13[train_index], data13[test_index]
    y13k_train, y13k_test = y[train_index], y[test_index]
    classifier_13.fit(x13k_train, y13k_train)
    y13k_pred = classifier_13.predict(x13k_test)
    kf_13_result.append(accuracy_score(y13k_test, y13k_pred))

# accumulated kfold result    
acc_result_13 = kf_13_result

#calculate the standard deviation and mean of the accuracy
print("the mean accuracy is: {}".format(mean(acc_result_13)))
print("the standard deviation of the accuracy is: {}".format(stdev(acc_result_13)))


the mean accuracy is: 0.5
the standard deviation of the accuracy is: 0.1875


In [15]:
# cross-validation for classifier23
kf_23 = KFold(n_splits=9, shuffle=True, random_state=32)

data23 = np.concatenate((data2, data3), axis = 0)
y2 = np.zeros(data1.shape[0])
y3 = np.ones(data1.shape[0])
y = np.concatenate((y2, y3), axis = 0)

kf_23_result = []
for train_index, test_index in kf_23.split(data23):
    x23k_train, x23k_test = data23[train_index], data23[test_index]
    y23k_train, y23k_test = y[train_index], y[test_index]
    classifier_23.fit(x23k_train, y23k_train)
    y23k_pred = classifier_23.predict(x23k_test)
    kf_23_result.append(accuracy_score(y23k_test, y23k_pred))

# accumulated kfold result    
acc_result_23 = kf_23_result

#calculate the standard deviation and mean of the accuracy
print("the mean accuracy is: {}".format(mean(acc_result_23)))
print("the standard deviation of the accuracy is: {}".format(stdev(acc_result_23)))


the mean accuracy is: 0.5
the standard deviation of the accuracy is: 0.22534695471649932


__5. Feature Importance using both 'recursive feature elimination' and 'permutation testing'__

In [18]:
# feature importance using recuresive feature elimination as feature selector
from sklearn.feature_selection import RFE

#the RFE wrapper method for estimating the features that contribute relatively to the classifier's prediction of a target
def recur_feat_elim(d1, d2):
    d1d2 = np.concatenate((d1, d2), axis = 0)
    y1y2 = np.concatenate((np.ones(data1.shape[0]) , np.zeros(data1.shape[0])), axis = 0)
    
    # to accurately estimate the feature importance we adopt one of the cross validation methods 
    #to prepare our model for prediction. RFE method selects the 15 best features.
    kf_cv = KFold(n_splits=9, shuffle=True, random_state=32)
    rfe_ranking = dict()
    classifier = LogisticRegression(penalty='l2', dual=False, tol=0.00001, C=1.0, fit_intercept=True, 
                                  intercept_scaling=1, class_weight='balanced', solver='newton-cg', max_iter=1000,
                                  multi_class='auto')
    for train_idx, test_idx in kf_cv.split(d1d2):
        rfe_model = RFE(classifier, n_features_to_select=15 )
        x_rfe_train, x_rfe_test = d1d2[train_idx], d1d2[test_idx]
        y_rfe_train, y_rfe_test = y1y2[train_idx], y1y2[test_idx]
        f_selector = rfe_model.fit(x_rfe_train, y_rfe_train)
        
        #we give ranking of 1 to the best 15 features selected 
        f_import_idx = np.where(f_selector.ranking_ == 1)[0]
        features_importance = feature_ids[f_import_idx]
        
        # for the input features in the dataset considered we determine how frequent the best features appear 
        for feat in range(features_importance.shape[0]):
            rfe_ranking[str(features_importance[feat])] = rfe_ranking.get(str(features_importance[feat]), 0) + 1
            
    return rfe_ranking

In [19]:
#find the 15 best features that have the highest frequency of appreance in the datasets considered
def feat_import(d1, d2):
    
    rfe_rank = recur_feat_elim(d1, d2)
    
    temp = rfe_rank.copy()
    test_val = -99999999
    rank = []
    
    for i in range(5):
        max_num = test_val
        for val in temp.keys():
            if(temp[val] > max_num):
                max_num = temp[val]
        for key, value in rfe_rank.items():
            if(value == max_num):
                rank.append(key)
                temp[key] = 0
    return rank


#Run the RFE feature selector method on the 3 classifiers


rank12 = feat_import(data1, data2)
print('features ranking in day1&2 datasets: ', rank12)

rank13 = feat_import(data1, data3)
print('features ranking in day1&3 datasets: ', rank13)

rank23 = feat_import(data2, data3)
print('features ranking in day2&3 datasets: ', rank23)

features ranking in day1&2 datasets:  ['[49 42]', '[49 50]', '[11 16]', '[11 22]', '[24 38]', '[18 42]', '[ 1 48]', '[46  4]', '[11 33]', '[53 42]', '[37 46]', '[56 46]', '[55 24]', '[40 51]']
features ranking in day1&3 datasets:  ['[49 33]', '[45 46]', '[ 1 11]', '[24 12]', '[ 3 22]', '[34 37]', '[26 47]', '[31  3]', '[ 5 24]', '[24 40]', '[19 43]', '[ 8 33]', '[23 52]', '[45  8]', '[21 20]', '[21 18]', '[45 52]', '[8 5]', '[46 42]', '[46  7]', '[46 36]', '[23 57]', '[51 41]', '[22  3]', '[ 6 22]', '[31 15]', '[11 38]', '[ 5 44]', '[36  9]', '[13 18]', '[35 18]', '[40 23]', '[23 30]', '[ 9 37]', '[47 42]', '[53 47]', '[11 21]', '[14 22]', '[11 35]', '[30  7]', '[60 10]', '[41 42]', '[13 28]', '[12 29]', '[11 43]', '[ 6 55]', '[26  4]', '[44 27]', '[ 3 41]', '[23 46]', '[11 55]', '[ 3 56]', '[23 17]', '[61 41]', '[49 42]', '[19 45]', '[ 5 19]', '[11 34]', '[53  9]', '[21 16]', '[10 24]', '[16 25]', '[15 35]', '[48 50]', '[7 1]', '[10 32]', '[34 48]', '[17 54]', '[12 10]', '[13 22]', '[

In [20]:
# features performance  with Permutation Testing method 
from sklearn.inspection import permutation_importance

def perm_import(d1, d2):
    d1d2 = np.concatenate((d1, d2), axis = 0)
    y1y2 = np.concatenate((np.ones(data1.shape[0]) , np.zeros(data1.shape[0])), axis = 0)
    
    # to accurately estimate the feature importance we adopt one of the cross validation methods 
    #to prepare our model for prediction. premutation importance method to determine the relatinoship between a feature an a target.
    kf_cv = KFold(n_splits=9, shuffle=True, random_state=32)
    PI_ranking = dict()
    classifier = LogisticRegression(penalty='l2', dual=False, tol=0.00001, C=1.0, fit_intercept=True, 
                                  intercept_scaling=1, class_weight='balanced', solver='newton-cg', max_iter=1000,
                                  multi_class='auto')
    for train_idx, test_idx in kf_cv.split(d1d2):
        x_pi_train, x_pi_test = d1d2[train_idx], d1d2[test_idx]
        y_pi_train, y_pi_test = y1y2[train_idx], y1y2[test_idx]
        model = classifier.fit(x_pi_train, y_pi_train)
        pi = permutation_importance(model, x_pi_test, y_pi_test, n_repeats=30)
        
        # permutation importance technique breaks the relationship between feature and target which reduces the model's accuracy.  
        for r in pi.importances_mean.argsort()[::-1]:
            if(pi.importances_mean[r] * 2 - r.importances_std[r] > 0):
                PI_ranking[str(feature_ids[r])] = PI_ranking.get(str(feature_ids[r]), 0) + 1
            
    return PI_ranking


In [21]:
# We want to see the level of improtance each feature that was severed from a target is.

def feat_import_PI(d1, d2):
    
    PI_rank = perm_import(d1, d2)
    
    temp = PI_rank.copy()
    test_val = -99999999
    rank = []
    
    for i in range(5):
        max_num = test_val
        for val in temp.keys():
            if(temp[val] > max_num):
                max_num = temp[val]
        for key, value in PI_rank.items():
            if(value == max_num):
                rank.append(key)
                temp[key] = 0
    return rank


#Run the Permutation importance method on the 3 classifiers

rank12 = feat_import(data1, data2)
print('features ranking in day1&2 datasets: ', rank12)

rank13 = feat_import(data1, data3)
print('features ranking in day1&3 datasets: ', rank13)

rank23 = feat_import(data2, data3)
print('features ranking in day2&3 datasets: ', rank23)

features ranking in day1&2 datasets:  ['[49 42]', '[49 50]', '[11 16]', '[11 22]', '[24 38]', '[18 42]', '[ 1 48]', '[46  4]', '[11 33]', '[53 42]', '[37 46]', '[56 46]', '[55 24]', '[40 51]']
features ranking in day1&3 datasets:  ['[49 33]', '[45 46]', '[ 1 11]', '[24 12]', '[ 3 22]', '[34 37]', '[26 47]', '[31  3]', '[ 5 24]', '[24 40]', '[19 43]', '[ 8 33]', '[23 52]', '[45  8]', '[21 20]', '[21 18]', '[45 52]', '[8 5]', '[46 42]', '[46  7]', '[46 36]', '[23 57]', '[51 41]', '[22  3]', '[ 6 22]', '[31 15]', '[11 38]', '[ 5 44]', '[36  9]', '[13 18]', '[35 18]', '[40 23]', '[23 30]', '[ 9 37]', '[47 42]', '[53 47]', '[11 21]', '[14 22]', '[11 35]', '[30  7]', '[60 10]', '[41 42]', '[13 28]', '[12 29]', '[11 43]', '[ 6 55]', '[26  4]', '[44 27]', '[ 3 41]', '[23 46]', '[11 55]', '[ 3 56]', '[23 17]', '[61 41]', '[49 42]', '[19 45]', '[ 5 19]', '[11 34]', '[53  9]', '[21 16]', '[10 24]', '[16 25]', '[15 35]', '[48 50]', '[7 1]', '[10 32]', '[34 48]', '[17 54]', '[12 10]', '[13 22]', '[